# Лабораторная работа №1

*Кищенко Екатерина  
ПИН-212*

## Задание 1
>Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).

Импорт библиотек

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

Загрузка датасета

In [7]:
df = pd.read_csv('metal.csv', encoding='utf-8')
df['Mode']= df['Mode'].map({1: True, 0: False}).astype(bool)
print(df.head(3))
print(df.dtypes)

               Spotify ID              Artist IDs Track Name  \
0  19vHgVS1aukRiQWhTqfKnE  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide   
1  5BgnL6gHauuvxe4Ok6W1aC  1Ffb6ejR6Fe5IamqA5oRUF       LosT   
2  0M3adYbGtyRHACP86dey1H  1Ffb6ejR6Fe5IamqA5oRUF     Throne   

          Album Name        Artist Name(s) Release Date  Duration (ms)  \
0           DArkSide  Bring Me The Horizon   10/18/2023         164790   
1               LosT  Bring Me The Horizon     5/4/2023         205368   
2  That's The Spirit  Bring Me The Horizon    9/11/2015         191520   

   Popularity                                Added By              Added At  \
0          74  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
1          72  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
2          76  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   

   ... Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0  ...   7    -3.018   True       0.1460      0.00035

Удаление ненужных столбцов из датасета

In [8]:
df = df.drop(['Spotify ID', 'Added By', 'Added At', 'Time Signature'], axis=1)
print(df.head(3))

               Artist IDs Track Name         Album Name        Artist Name(s)  \
0  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide           DArkSide  Bring Me The Horizon   
1  1Ffb6ejR6Fe5IamqA5oRUF       LosT               LosT  Bring Me The Horizon   
2  1Ffb6ejR6Fe5IamqA5oRUF     Throne  That's The Spirit  Bring Me The Horizon   

  Release Date  Duration (ms)  Popularity  \
0   10/18/2023         164790          74   
1     5/4/2023         205368          72   
2    9/11/2015         191520          76   

                                              Genres  Danceability  Energy  \
0  melodic metalcore,metalcore,modern rock,rock,u...         0.387   0.940   
1  melodic metalcore,metalcore,modern rock,rock,u...         0.396   0.939   
2  melodic metalcore,metalcore,modern rock,rock,u...         0.426   0.888   

   Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0    7    -3.018   True       0.1460      0.000352          0.000003   
1    9    -3.155  False       0.2270 

Удаление строк с пропущенными значениями

In [9]:
df.dropna(inplace=True)

Предобработка данных

In [10]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
print(df.head(3))
print(df.dtypes)

   Artist IDs  Track Name  Album Name  Artist Name(s)  Release Date  \
0         212        2152         631             182           206   
1         212        5399        1647             182          1178   
2         212        9575        2696             182          1740   

   Duration (ms)  Popularity  Genres  Danceability  Energy  Key  Loudness  \
0         164790          74     972         0.387   0.940    7    -3.018   
1         205368          72     972         0.396   0.939    9    -3.155   
2         191520          76     972         0.426   0.888    0    -3.720   

    Mode  Speechiness  Acousticness  Instrumentalness  Liveness  Valence  \
0   True       0.1460      0.000352          0.000003     0.628    0.456   
1  False       0.2270      0.000285          0.000000     0.212    0.390   
2  False       0.0987      0.000455          0.000000     0.306    0.387   

     Tempo  
0  167.910  
1  185.081  
2  144.111  
Artist IDs            int32
Track Name           

In [11]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])
print(df.head(3))

   Artist IDs  Track Name  Album Name  Artist Name(s)  Release Date  \
0   -1.032943   -1.047387   -1.132416       -1.214946     -1.191165   
1   -1.032943   -0.007295   -0.120908       -1.214946      0.549409   
2   -1.032943    1.330378    0.923455       -1.214946      1.555790   

   Duration (ms)  Popularity    Genres  Danceability    Energy       Key  \
0      -0.919780    2.410398  1.890198      0.252167  0.382949  0.498957   
1      -0.548958    2.298142  1.890198      0.317669  0.377046  1.066209   
2      -0.675508    2.522654  1.890198      0.536006  0.076001 -1.486428   

   Loudness   Mode  Speechiness  Acousticness  Instrumentalness  Liveness  \
0  1.034060   True     0.874084     -0.227497         -0.943709  2.487116   
1  0.992878  False     2.215229     -0.227993         -0.943718 -0.051936   
2  0.823040  False     0.090922     -0.226736         -0.943718  0.521792   

    Valence     Tempo  
0  1.007971  1.476925  
1  0.650282  2.060858  
2  0.634023  0.667594  


Разделение данных на обучающий и тестовый наборы

In [12]:
X = df.drop('Mode', axis=1)
Y = df['Mode']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Задание 2  
>Провести классификацию найденного датасета, методом k-ближайших соседей.

Обучение модели k-ближайших соседей

In [26]:
# Определение сетки параметров для поиска
param_grid = {'n_neighbors': [5, 7, 9, 11, 13]}
parameters = {'metric':('hamming', 'minkowski', 'manhattan', 'braycurtis', 'correlation', 'nan_euclidean', 'cityblock', 'euclidean'), 'n_neighbors':[5, 7, 9, 11, 13]}

Создание модели k-ближайших соседей

In [27]:
knn = KNeighborsClassifier()

Поиск оптимальных параметров с использованием кросс-валидации

In [28]:
grid_search = GridSearchCV(knn, parameters, cv=3)  # cv - количество фолдов для кросс-валидации
grid_search.fit(X_train, Y_train)

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\metrics\_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 705, in score
    return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
                             ^^^^^^^^^^^^^^^
  File "C:\Anaconda\Lib\site-packages\sklearn\neighbors\_classification.py", line 246, in predict
    if self._fit_method == "brute" and ArgKminClassMode.is_usable_for(
                                      

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'metric': ('hamming', 'minkowski', 'manhattan',
                                    'braycurtis', 'correlation',
                                    'nan_euclidean', 'cityblock', 'euclidean'),
                         'n_neighbors': [5, 7, 9, 11, 13]})

Получение лучших параметров

In [29]:
best_parametrs = grid_search.best_params_
print(best_parametrs)

{'metric': 'hamming', 'n_neighbors': 5}


Обучение модели с лучшими параметрами

In [30]:
best_knn = grid_search.best_estimator_
best_knn.fit(X_train, Y_train)

KNeighborsClassifier(metric='hamming')

Предсказание на тестовом наборе данных

In [35]:
Y_pred = best_knn.predict(X_test.values)

C:\Anaconda\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Оценка модели

In [37]:
result = classification_report(Y_test, Y_pred)

print(result)

              precision    recall  f1-score   support

       False       0.53      0.44      0.48       991
        True       0.69      0.76      0.72      1601

    accuracy                           0.64      2592
   macro avg       0.61      0.60      0.60      2592
weighted avg       0.63      0.64      0.63      2592



**1. Точность (Precision)**  
Точность показывает долю правильных положительных предсказаний среди всех предсказанных положительных примеров.  
Высокая точность означает, что модель имеет низкое количество ложных положительных предсказаний.  

**2. Полнота (Recall)**  
Полнота показывает долю правильных положительных предсказаний среди всех реальных положительных примеров.  
Высокая полнота означает, что модель имеет низкое количество ложных отрицательных предсказаний.  

**3. F1-мера (F1-score)**  
F1-мера является гармоническим средним между точностью и полнотой. Эта метрика полезна, когда требуется балансировать между точностью и полнотой.  
Высокое значение F1-меры показывает, что модель хорошо справляется с балансом между точностью и полнотой.  

**4. Поддержка (Support)**  
Поддержка показывает количество истинных примеров каждого класса в тестовом наборе данных.  
Высокая поддержка говорит о большом количестве примеров данного класса, что делает оценку метрик более стабильной.

>**precision** для класса False: 0.53 означает, что 53% предсказанных положительных примеров для класса False были правильными.  
**recall** для класса False: 0.44 означает, что 44% реальных положительных примеров для класса False были правильно предсказаны.   
**f1-score** для класса False: 0.48 является гармоническим средним между точностью и полнотой.  
**support** для класса False: 991 означает, что в тестовом наборе данных был 991 пример класса False.